#                                      Prediksi Perkiraan Penjualan

## import SparkSession 

In [11]:
import os
import findspark
findspark.init()
import warnings
warnings.filterwarnings('ignore')
from pyspark.sql import SparkSession
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, FloatType
from pyspark.sql.functions import split, count, when, isnan, col, regexp_replace
from pyspark.sql import Row

In [12]:
# Inisialisasi sesi Spark
spark = SparkSession.builder.appName("SalesPrediction").getOrCreate()

## Membuat Data Frame nya menggunakan Numpy, Tidak menggunakan Data Set CSV

In [13]:
import numpy as np

In [14]:
# Buat dataset dasar
data = []
np.random.seed(0)

for _ in range(100):
    harga = np.random.randint(50, 500)
    iklan = np.random.randint(1, 10)
    populeritas = np.random.randint(10, 100)
    persediaan = np.random.randint(10, 100)
    penjualan = 1000 + (10 * harga) - (5 * iklan) + (2 * populeritas) + (3 * persediaan)
    data.append(Row(harga=harga, iklan=iklan, populeritas=populeritas, persediaan=persediaan, penjualan=penjualan))

In [15]:

# Buat DataFrame dari data
df = spark.createDataFrame(data)

## Setelah data dibuat lalu menampilkan data dengan top 10

In [16]:
# Tampilkan 10 baris data teratas
df.show(10)


+-----+-----+-----------+----------+---------+
|harga|iklan|populeritas|persediaan|penjualan|
+-----+-----+-----------+----------+---------+
|  222|    6|         74|        77|     3569|
|  301|    4|         19|        93|     4307|
|  327|    3|         46|        97|     4638|
|  120|    9|         98|        22|     2417|
|  364|    2|         49|        97|     5019|
|  224|    9|         91|        47|     3518|
|   75|    9|         19|        30|     1833|
|  165|    1|         79|        89|     3070|
|  481|    1|         92|        98|     6283|
|  227|    4|         39|        29|     3415|
+-----+-----+-----------+----------+---------+
only showing top 10 rows



In [17]:
df.printSchema()

root
 |-- harga: long (nullable = true)
 |-- iklan: long (nullable = true)
 |-- populeritas: long (nullable = true)
 |-- persediaan: long (nullable = true)
 |-- penjualan: long (nullable = true)



### Modul ini menyediakan alat untuk mengelola data vektor.
##### Vektor digunakan untuk menggabungkan fitur-fitur data ke dalam satu entitas yang dapat digunakan oleh model machine learning. 

In [18]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [19]:
# Buat vektor fitur
assembler = VectorAssembler(inputCols=["harga", "iklan", "populeritas", "persediaan"], outputCol="fitur")
df = assembler.transform(df)

In [20]:

# Pisahkan data menjadi data latih dan data uji
train_data, test_data = df.randomSplit([0.8, 0.2], seed=42)

#### LinearRegression adalah salah satu algoritma regresi yang tersedia dalam Apache Spark's MLlib (Machine Learning Library)

In [21]:
from pyspark.ml.regression import LinearRegression

### Regresi linear adalah teknik statistik yang digunakan untuk memodelkan hubungan antara variabel dependen (dalam hal ini, "penjualan") dan satu atau lebih variabel independen (fitur-fitur seperti "harga," "iklan," "populeritas," dan "persediaan").

In [22]:
# Inisialisasi model regresi linear
lr = LinearRegression(featuresCol="fitur", labelCol="penjualan")

In [23]:
# Latih model
model = lr.fit(train_data)

In [24]:
# Lakukan prediksi pada data uji
predictions = model.transform(test_data)

In [25]:
# Tampilkan beberapa prediksi
predictions.select("harga", "iklan", "populeritas", "persediaan", "penjualan", "prediction").show()

+-----+-----+-----------+----------+---------+------------------+
|harga|iklan|populeritas|persediaan|penjualan|        prediction|
+-----+-----+-----------+----------+---------+------------------+
|  132|    1|         24|        63|     2552|2552.0000000000036|
|  198|    4|         68|        33|     3195|3195.0000000000014|
|  224|    9|         91|        47|     3518|3517.9999999999995|
|  312|    5|         57|        13|     4248|            4248.0|
|  420|    8|         38|        44|     5368| 5367.999999999999|
|  475|    4|         21|        56|     5940| 5939.999999999999|
|   92|    4|         66|        70|     2242| 2242.000000000002|
|  144|    3|         23|        50|     2621| 2621.000000000003|
|  148|    1|         31|        83|     2786|2786.0000000000023|
|  298|    5|         77|        71|     4322| 4321.999999999999|
|  357|    1|         42|        64|     4841|            4841.0|
|  433|    1|         20|        53|     5524| 5523.999999999999|
|   79|   

### Nilai RMSE yang sangat kecil, seperti "1.911156558815751e-12," yang sangat mendekati nol,
### menunjukkan bahwa model Anda hampir sempurna dalam memprediksi data penjualan.
### Dalam kasus ini, nilai prediksi model hampir sama persis dengan nilai sebenarnya.
### Ini adalah hasil yang sangat baik dan mengindikasikan bahwa model Anda sangat akurat.

In [26]:
# Evaluasi model (misalnya, menggunakan RMSE)
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(labelCol="penjualan", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE):", rmse)

# Stop sesi Spark
spark.stop()


Root Mean Squared Error (RMSE): 1.911156558815751e-12
